In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extrair_subopcoes(conteudo_html):
    sopa = BeautifulSoup(conteudo_html, 'html.parser')
    botoes = sopa.find_all('button', class_='btn_sopt')
    mapeamento_subopcao = {}
    for botao in botoes:
        subopcao = botao['value']
        nome_produto = botao.get_text(strip=True)
        mapeamento_subopcao[subopcao] = nome_produto
    return mapeamento_subopcao

In [5]:


def raspar_viti_brasil_processamento(ano, opcao):
    dicionario_opcao = {'opt_03': 'Processamento'}  # Defina dicionario_opcao dentro da função

    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao={opcao}'
    resposta = requests.get(url)
    
    if resposta.status_code == 200:
        mapeamento_subopcao = extrair_subopcoes(resposta.content)
        lista_dfs = []
        for subopcao, produto in mapeamento_subopcao.items():
            sub_url = f'{url}&subopcao={subopcao}'
            sub_resposta = requests.get(sub_url)
            if sub_resposta.status_code == 200:
                sub_sopa = BeautifulSoup(sub_resposta.content, 'html.parser')
                tabela = sub_sopa.find('table', class_='tb_dados')
                if tabela:
                    dados = []
                    linhas = tabela.find_all('tr')
                    for linha in linhas[1:]:
                        celulas = linha.find_all('td')
                        if celulas:
                            if 'tb_item' in celulas[0].get('class', []):
                                produto = celulas[0].get_text(strip=True)
                                quantidade = celulas[1].get_text(strip=True)
                                dados.append({'tabela': dicionario_opcao[opcao], 'produto': produto, 'sub_produto': None, 'quantidade': quantidade, 'ano': ano, 'unidade_medida': 'kg'})
                            elif 'tb_subitem' in celulas[0].get('class', []):
                                sub_produto = celulas[0].get_text(strip=True)
                                quantidade = celulas[1].get_text(strip=True)
                                dados.append({'tabela': dicionario_opcao[opcao], 'produto': produto, 'sub_produto': sub_produto, 'quantidade': quantidade, 'ano': ano, 'unidade_medida': 'kg'})
                    df = pd.DataFrame(dados)
                    df = df.dropna(subset=['sub_produto'])
                    lista_dfs.append(df)
                    
                else:
                    print(f'Tabela não encontrada para subopção {subopcao}.')
            else:
                print(f'Falha ao fazer a solicitação HTTP para subopcao {subopcao}.')
        
        if lista_dfs:
            return pd.concat(lista_dfs, ignore_index=True)
        else:
            return None
    else:
        print('Falha ao fazer a solicitação HTTP para a página principal.')
        return None

# Exemplo de uso da função
ano = 2021
opcao = 'opt_03'
resultado = raspar_viti_brasil_processamento(ano, opcao)
if resultado is not None:
    print(resultado)

Nenhum dado encontrado.
